In [1]:
import os
import sys
import allel
import subprocess
from tqdm import tqdm_notebook as tqdm

sys.path.append("mutect2/")
# mutect2 -- bed,filter_mutect2, run_mutect2
sys.path.append("rvb_vs_mut2/")

<center><h4> Делаем список генов для каждой пары опухоль-норма </h4></center>

In [3]:
import bed

In [4]:
for i in [16]:
    for j in [1,2]:
        bed.make_bed_file(i,j)

In [11]:
# BAM="/media/emir/Seagate/data/patients"
# path_to_output="/media/emir/Storage/Cancer/mutect/output"

# for i in os.listdir(BAM):
#     a="pat"+i.split("_")[1]
#     os.mkdir(os.path.join(path_to_output, a, "s2"))

<center><h4> Запускаем mutect2 и фильтруем результаты </center></h4>

In [2]:
import run_mutect2
import filter_mutect2

In [3]:
for i in [16]:
    for j in tqdm([1,2]):
        run_mutect2.search_mutations(i,j,1)

In [5]:
for i in [16]:
    for j in tqdm([1,2]):
        filter_mutect2.filter_somatic(i,j)

<center><h4>Аннотируем варианты с помощью snpEff</h4></center>

While PolyPhen-2 HDIV uses alleles encoding human proteins and their closely related mammalian homologs as TN observations, PolyPhen-2 HVAR applies common human nsSNVs as TN observations. 

In [2]:
import annotate
import comparison

In [7]:
for i in [16]:
    for j in tqdm([1,2]):
        annotate.annotate_variants(i,j)

In [8]:
for i in [16]:
    for j in tqdm([1,2]):
        annotate.func_pred(i,j)

In [16]:
for i in [1,2,3,4,"T"]:
    for j in tqdm([1,2]):
        annotate.dbsnp_annotate(i,j)

In [31]:
# def add_dbsnp(patient,sample):
#     path_to_mutect2 = f"/media/emir/Storage/Cancer/mutect/output/pat{patient}/s{sample}"
#     path_to_vcf = f"/media/emir/Storage/Cancer/mutect/output/pat{patient}/s{sample}/output_transcr_new_predicted_dbsnp.vcf"
#     mutect2_name = "mutect2_w_dbsnp.pkl"
#     df=pd.read_pickle(os.path.join(path_to_mutect2,mutect2_name))
#     df_vcf=allel.vcf_to_dataframe(path_to_vcf,fields="ID")
#     df.insert(2,"ID",df_vcf)
#     df.to_pickle(os.path.join(path_to_mutect2,"mutect2_w_dbsnp.pkl"))
#     return df 

In [4]:
for i in [16]:
    for j in tqdm([1,2]):
        comparison.rvboost_mutect2(i,j,1)

<Figure size 432x288 with 0 Axes>

<center><h4> Делаем списки всех тестируемых генов для графини (old) </center></h4>

In [11]:
def grafinya(patient):
    
    output_dir=f"/media/emir/Storage/Cancer/mutect/output/pat{patient}"
    file_name=f"genes_intersect_pat{patient}"
    df=comparison.mutect2_s1_s2(patient)
    gene_list=df["ANN_Gene_Name_x"].unique().tolist()
    print(len(gene_list))
    with open(os.path.join(output_dir,file_name), "w") as s:
        for i in gene_list:
            if isinstance(i,float):
                gene_list.remove(i)
            else:
                s.write(i+"\n")
                
    print(len(gene_list))


<center><h4> Предположение о поломанных bam файлах anzaika (1.1, 3.1, 38.1) </center></h4>

In [150]:
def samtools_head(patient, sample):
    
    result=subprocess.run("BAM=/media/emir/Seagate/data/patients; \
                samtools view -H $BAM/Patient_{0}/Tumor_{1}/reorder.sorted.bam".format(patient,sample), 
                shell = True, stdout=subprocess.PIPE)
        
    return print(result.stdout.decode('utf-8'))

In [153]:
def samtools(patient, sample):

    result=subprocess.run("BAM=/media/emir/Seagate/data/patients; \
                samtools depth -r chr1:100000-1000000 $BAM/Patient_{0}/Tumor_{1}/reorder.sorted.bam".format(patient,sample), 
                shell = True, stdout=subprocess.PIPE)
    l=result.stdout.decode('utf-8').split("\n")
    c=0
    for i in l[0:len(l)-1]:
        dc=i.split("\t")[2]
        c+=int(dc)
    return c

In [155]:
samtools("T",2)/samtools("T",1)

1.1318716496879468

In [150]:
5.5/1.9

2.8947368421052633

1_1, 3_1, 38_1 -- делала anzaika -- для всех них нашлось крайне мало мутаций как rvboost, так и mutect2

<center><h4> Пересекаем результаты rvboost и mutect2 </center></h4>

In [2]:
import comparison
from tqdm import tqdm_notebook as tqdm

In [4]:
for i in ["T"]:
    for j in tqdm([1,2]):
        comparison.rvboost_mutect2(i,j,1)

<Figure size 432x288 with 0 Axes>

In [3]:
# comparison.save_pics("T")
# comparison.show_pics("T")

<center><h4> Делаем excel таблицу для пациента </center></h4>

Листы вида: <br>
all_muts_s{sample} -- все найденные mutect2 мутации <br>
mut2_exclusive_muts_s{sample} -- нашлись только mutect2 <br>
commn_muts_for_rvb_mut2_s{sample} -- общие для выдачи mutect2 и rvboost

<center><h4> Сравниваем результаты mutect2 с кодирующими космиковскими мутациями </center></h4>

In [5]:
# comparison.mutect2_s1_s2(1)

Для 1,1: <br>
polyphen2_hdiv + cosmic + impact -- 20 из 100<br>
polyphen2_hvar + cosmic + impact -- 18 из 96<br>
sift + cosmic + impact -- 27 из 150<br>
metalr + cosmic + impact -- 11 из 58<br>
mutationtaster_a + cosmic + impact -- 12 из 12<br>
mutationtaster_d + cosmic + impact -- 35 из 207<br>

Картина с mutationtaster_a не такая радужная у других пациентов

<center><h4> Сколько мутаций общие для космиковских rvboost и mutect2 </center></h4>

In [18]:
# comparison.compare_cosmic("T",1)

<center><h4>Какой процент мутаций с импактом есть в космике</center></h4>

In [32]:
l=[]
for i in [1,2,3,4,"T"]:
    for j in [1,2]:
        a=comparison.compare_to_cosmic(i,j)
        l.append(a[1]/a[0])
print (l)

[0.39622641509433965, 0.36542443064182195, 0.5778210116731517, 0.6416739319965127, 0.42783505154639173, 0.29615861214374223, 0.5297805642633229, 0.37514654161781946, 0.6750418760469011, 0.5826271186440678]


<center><h4>Какой процент мутаций с импактом и filter pass есть в космике</center></h4>

In [34]:
l=[]
for i in [1,2,3,4,"T"]:
    for j in [1,2]:
        a=comparison.compare_to_cosmic(i,j)
        l.append(a[1]/a[0])
print (l)

[0.25146198830409355, 0.22285714285714286, 0.3695652173913043, 0.5487465181058496, 0.18181818181818182, 0.2049586776859504, 0.3689727463312369, 0.24285714285714285, 0.43478260869565216, 0.33884297520661155]


Если смотрим на импакт и filter_pass -- цифры меньше. Значит много соматических космиковских мутаций не проходят по той или иной причине фильтры mutect2.

Вывод: просто смотрим на импакт и сравниваем с космиком.

<center><h4>Какой процент мутаций с импактом есть в космике (результаты rvboost)</center></h4>

In [38]:
l=[]
for i in [1,2,3,4,"T"]:
    for j in [1,2]:
        a=comparison.compare_to_cosmic(i,j,1)
        l.append(a[1]/a[0])
print (l)

[0.5707547169811321, 0.5794143744454303, 0.6073446327683616, 0.4196204797708557, 0.615411681914145, 0.5917094644167278, 0.42509777248767217, 0.40334638661445354, 0.6291593695271454, 0.3740443781465598]


В 7 из 10 случаев rvboost нашел больше космиковских мутаций, чем mutect2.

<center><h4> Фильтруем мутации </center></h4>

In [4]:
for i in [1,2,3,4,16,"T"]:
    for j in tqdm([1,2]):
        comparison.filter_merged_muts(i,j)
# comparison.filter_merged_muts(1,1)